In [4]:
from numpy.random import seed
from tensorflow import set_random_seed
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras import backend as K
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import numpy as np
import tensorflow as tf
import random as python_random
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import r2_score
from pandas import DataFrame
from itertools import repeat

In [45]:
#Set seeds so neural network is repeatable
#Training and results on entire dataset

#Seeding details sourced from https://medium.com/@ODSC/properly-setting-the-random-seed-in-ml-experiments-not-as-simple-as-you-might-imagine-219969c84752

seed_value= 123

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

#Implementation of r2 in Keras. Code snippet sourced from https://jmlb.github.io/ml/2017/03/20/CoeffDetermination_CustomMetric4Keras/
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

dependent = range(11,18)

print("\nBMO")

dataframe = pd.read_csv("BMO.csv")

r2_dependent=[]
mse_dependent=[]

for num in dependent:
    num1 = num-10
    print("\nDependent #"+str(num1))

    r2=[]
    mse=[]
    count=1
    for i in repeat(None, 5):
        X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])

        model = Sequential()
        model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
        start_time = time.time()

        model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)
        end_time = time.time()
        modeltime = end_time-start_time

        score = model.evaluate(X_test, y_test,verbose=0)

        print("\nfold number: " +str(count))
        print("%s: %f" % ("Mean Squared Error", score[0]))
        print("%s: %f" % ("R Squared", score[1]))
        print("model time: %f" % modeltime)
        mse.append(score[0])
        r2.append(score[1])
        count=count+1
    r2_dependent.append(np.mean(r2))
    mse_dependent.append(np.mean(mse))
    print("\nTotal")    
    print(np.mean(r2))
    print(np.mean(mse))
bmo_neural1_r2 = DataFrame(r2_dependent).transpose()
bmo_neural1_r2.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
bmo_neural1_r2['Bank']="BMO"
bmo_neural1_mse = DataFrame(mse_dependent).transpose()
bmo_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
bmo_neural1_mse['Bank']="BMO"

print("\nCIBC")

dataframe = pd.read_csv("CIBC.csv")
#dataframe = dataframe[dataframe['Quarter']==1]
r2_dependent=[]
mse_dependent=[]

for num in dependent:
    num1 = num-10
    print("\nDependent #"+str(num1))

    r2=[]
    mse=[]
    count=1
    for i in repeat(None, 5):
        X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])

        model = Sequential()
        model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
        start_time = time.time()

        model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)
        end_time = time.time()
        modeltime = end_time-start_time

        score = model.evaluate(X_test, y_test,verbose=0)
        #y_pred = model.predict(X_test)
        #score1 = r2_score(y_test,y_pred)

        print("\nfold number: " +str(count))
        print("%s: %f" % ("Mean Squared Error", score[0]))
        print("%s: %f" % ("R Squared", score[1]))
        print("model time: %f" % modeltime)
        mse.append(score[0])
        r2.append(score[1])
        count=count+1
    r2_dependent.append(np.mean(r2))
    mse_dependent.append(np.mean(mse))
    print("\nTotal")    
    print(np.mean(r2))
    print(np.mean(mse))
cibc_neural1_r2 = DataFrame(r2_dependent).transpose()
cibc_neural1_r2.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
cibc_neural1_r2['Bank']="CIBC"
cibc_neural1_mse = DataFrame(mse_dependent).transpose()
cibc_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
cibc_neural1_mse['Bank']="CIBC"

print("\nTD")

dataframe = pd.read_csv("TD.csv")

r2_dependent=[]
mse_dependent=[]

for num in dependent:
    num1 = num-10
    print("\nDependent #"+str(num1))

    r2=[]
    mse=[]
    count=1
    for i in repeat(None, 5):
        X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])

        model = Sequential()
        model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
        start_time = time.time()

        model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)
        end_time = time.time()
        modeltime = end_time-start_time

        score = model.evaluate(X_test, y_test,verbose=0)

        print("\nfold number: " +str(count))
        print("%s: %f" % ("Mean Squared Error", score[0]))
        print("%s: %f" % ("R Squared", score[1]))
        print("model time: %f" % modeltime)
        mse.append(score[0])
        r2.append(score[1])
        count=count+1
    r2_dependent.append(np.mean(r2))
    mse_dependent.append(np.mean(mse))
    print("\nTotal")    
    print(np.mean(r2))
    print(np.mean(mse))
td_neural1_r2 = DataFrame(r2_dependent).transpose()
td_neural1_r2.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
td_neural1_r2['Bank']="TD"
td_neural1_mse = DataFrame(mse_dependent).transpose()
td_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
td_neural1_mse['Bank']="TD"

print("\nRBC")

dataframe = pd.read_csv("RBC.csv")

r2_dependent=[]
mse_dependent=[]

for num in dependent:
    num1 = num-10
    print("\nDependent #"+str(num1))


    r2=[]
    mse=[]
    count=1
    for i in repeat(None, 5):
        X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])

        model = Sequential()
        model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
        start_time = time.time()

        model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)
        end_time = time.time()
        modeltime = end_time-start_time

        score = model.evaluate(X_test, y_test,verbose=0)

        print("\nfold number: " +str(count))
        print("%s: %f" % ("Mean Squared Error", score[0]))
        print("%s: %f" % ("R Squared", score[1]))
        print("model time: %f" % modeltime)
        mse.append(score[0])
        r2.append(score[1])
        count=count+1
    r2_dependent.append(np.mean(r2))
    mse_dependent.append(np.mean(mse))
    print("\nTotal")    
    print(np.mean(r2))
    print(np.mean(mse))
rbc_neural1_r2 = DataFrame(r2_dependent).transpose()
rbc_neural1_r2.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
rbc_neural1_r2['Bank']="RBC"
rbc_neural1_mse = DataFrame(mse_dependent).transpose()
rbc_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
rbc_neural1_mse['Bank']="RBC"

print("\nScotia")

dataframe = pd.read_csv("scotia.csv")

r2_dependent=[]
mse_dependent=[]

for num in dependent:
    num1 = num-10
    print("\nDependent #"+str(num1))


    r2=[]
    mse=[]
    count=1
    for i in repeat(None, 5):
        X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])

        model = Sequential()
        model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
        model.add(Dense(1, kernel_initializer='normal'))
        # Compile model
        model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
        start_time = time.time()

        model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)
        end_time = time.time()
        modeltime = end_time-start_time

        score = model.evaluate(X_test, y_test,verbose=0)
        #y_pred = model.predict(X_test)
        #score1 = r2_score(y_test,y_pred)

        print("\nfold number: " +str(count))
        print("%s: %f" % ("Mean Squared Error", score[0]))
        print("%s: %f" % ("R Squared", score[1]))
        print("model time: %f" % modeltime)
        mse.append(score[0])
        r2.append(score[1])
        count=count+1
    r2_dependent.append(np.mean(r2))
    mse_dependent.append(np.mean(mse))
    print("\nTotal")    
    print(np.mean(r2))
    print(np.mean(mse))
scotia_neural1_r2 = DataFrame(r2_dependent).transpose()
scotia_neural1_r2.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
scotia_neural1_r2['Bank']="Scotia"
scotia_neural1_mse = DataFrame(mse_dependent).transpose()
scotia_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
scotia_neural1_mse['Bank']="Scotia"


BMO

Dependent #1


Exception ignored in: <bound method ScopedTFStatus.__del__ of <tensorflow.python.framework.c_api_util.ScopedTFStatus object at 0x00000297B567DFD0>>
Traceback (most recent call last):
  File "C:\Users\roryc\Anaconda3\lib\site-packages\tensorflow\python\framework\c_api_util.py", line 39, in __del__
    c_api.TF_DeleteStatus(self.status)
AttributeError: 'ScopedTFStatus' object has no attribute 'status'



fold number: 1
Mean Squared Error: 0.000000
R Squared: -0.915255
model time: 108.434712

fold number: 2
Mean Squared Error: 0.000000
R Squared: -0.482095
model time: 103.270721

Total
-0.6986748278141022
7.917431618157263e-08

Dependent #2

fold number: 1
Mean Squared Error: 0.000406
R Squared: 0.222898
model time: 103.982034

fold number: 2
Mean Squared Error: 0.000407
R Squared: 0.228258
model time: 105.538248

Total
0.2255784049630165
0.0004068228021439569

Dependent #3

fold number: 1
Mean Squared Error: 0.000561
R Squared: 0.199111
model time: 104.051668

fold number: 2
Mean Squared Error: 0.000556
R Squared: 0.197318
model time: 105.123293

Total
0.19821488857269287
0.0005586361798233995

Dependent #4

fold number: 1
Mean Squared Error: 0.000000
R Squared: -0.346408
model time: 108.105404

fold number: 2
Mean Squared Error: 0.000000
R Squared: 0.091214
model time: 106.353576

Total
-0.12759703770279884
7.549464943093381e-08

Dependent #5

fold number: 1
Mean Squared Error: 0.000


fold number: 2
Mean Squared Error: 0.000423
R Squared: 0.491767
model time: 83.661840

Total
0.5042324513196945
0.0004129803212710759


In [17]:
#Set seeds so neural network results are repeatable
#Training on entire dataset, testing done quarterly
seed_value= 123

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.compat.v1.set_random_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)


dependent = range(11,18)
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

print("CIBC")

r2_list=[]
mse_list=[]

for num in dependent:
    num1 = num-10

    dataframe = pd.read_csv("CIBC.csv")

    r2=[]
    mse=[]
    X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])

    model = Sequential()
    model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
    model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)
    #model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=1)

    for i in range(32):
        #print("Quarter#"+str(i+1))
        dataframe = pd.read_csv("CIBC.csv")
        dataframe = dataframe[dataframe['Quarter']==i+1]
        X = dataframe.iloc[:,5:11]
        Y = dataframe.iloc[:,num]

        score = model.evaluate(X, Y,verbose=0)
        #print("\n%s: %f" % ("Mean Squared Error", score[0]))
        mse.append(score[0])
    mse_list.append(mse)
cibc_neural1_mse = DataFrame(mse_list).transpose()
cibc_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
cibc_neural1_mse['Bank']="CIBC"

print("BMO")

r2_list=[]
mse_list=[]

for num in dependent:
    num1 = num-10

    dataframe = pd.read_csv("BMO.csv")

    r2=[]
    mse=[]
    X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])

    model = Sequential()
    model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
    model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)


    for i in range(32):
        #print("Quarter#"+str(i+1))
        dataframe = pd.read_csv("BMO.csv")
        dataframe = dataframe[dataframe['Quarter']==i+1]
        X = dataframe.iloc[:,5:11]
        Y = dataframe.iloc[:,num]

        score = model.evaluate(X, Y,verbose=0)

        mse.append(score[0])
    mse_list.append(mse)
bmo_neural1_mse = DataFrame(mse_list).transpose()
bmo_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
bmo_neural1_mse['Bank']="BMO"

print("TD")

r2_list=[]
mse_list=[]

for num in dependent:
    num1 = num-10
    print("\nDependent #"+str(num1))

    dataframe = pd.read_csv("TD.csv")

    r2=[]
    mse=[]
    X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])

    model = Sequential()
    model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
    model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)
    #model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=1)

    for i in range(32):
        dataframe = pd.read_csv("TD.csv")
        dataframe = dataframe[dataframe['Quarter']==i+1]
        X = dataframe.iloc[:,5:11]
        Y = dataframe.iloc[:,num]

        score = model.evaluate(X, Y,verbose=0)
        mse.append(score[0])
    mse_list.append(mse)
td_neural1_mse = DataFrame(mse_list).transpose()
td_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
td_neural1_mse['Bank']="TD"

print("RBC")

r2_list=[]
mse_list=[]

for num in dependent:
    num1 = num-10

    dataframe = pd.read_csv("RBC.csv")

    r2=[]
    mse=[]
    X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])
  
    model = Sequential()
    model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
    model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)
    #model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=1)

    for i in range(32):
        dataframe = pd.read_csv("RBC.csv")
        dataframe = dataframe[dataframe['Quarter']==i+1]
        X = dataframe.iloc[:,5:11]
        Y = dataframe.iloc[:,num]

        score = model.evaluate(X, Y,verbose=0)
        mse.append(score[0])
    mse_list.append(mse)
rbc_neural1_mse = DataFrame(mse_list).transpose()
rbc_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
rbc_neural1_mse['Bank']="RBC"

print("Scotia")

r2_list=[]
mse_list=[]

for num in dependent:
    num1 = num-10

    dataframe = pd.read_csv("scotia.csv")
    #dataframe = dataframe[dataframe['Quarter']==1]
    r2=[]
    mse=[]
    X_train,X_test,y_train,y_test = train_test_split(dataframe.iloc[:,5:11],dataframe.iloc[:,num],test_size = 0.2,stratify=dataframe["Quarter"])

    model = Sequential()
    model.add(Dense(6, input_dim=6, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=[coeff_determination])
    model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=0)
    #model.fit(X_train, y_train, epochs=200, batch_size=96,verbose=1)

    for i in range(32):

        dataframe = pd.read_csv("scotia.csv")
        dataframe = dataframe[dataframe['Quarter']==i+1]
        X = dataframe.iloc[:,5:11]
        Y = dataframe.iloc[:,num]

        score = model.evaluate(X, Y,verbose=0)

        mse.append(score[0])
    mse_list.append(mse)
scotia_neural1_mse = DataFrame(mse_list).transpose()
scotia_neural1_mse.columns = ['ROA','ROS','Operating Income to Sales','Opearting Income to Assets','Operating Expenses to Sales','Cost of Goods to Sales','Selling, General and Administrative Expenses to Sales']
scotia_neural1_mse['Bank']="Scotia"

cibc_neural1_mse
bmo_neural1_mse
rbc_neural1_mse
td_neural1_mse
scotia_neural1_mse
Neural1_mse = pd.concat([cibc_neural1_mse,bmo_neural1_mse,rbc_neural1_mse,td_neural1_mse,scotia_neural1_mse])
Neural1_mse.to_csv('cibc1.csv', index=False)

CIBC

Dependent #1

Dependent #2

Dependent #3

Dependent #4

Dependent #5

Dependent #6

Dependent #7
BMO

Dependent #1

Dependent #2

Dependent #3

Dependent #4

Dependent #5

Dependent #6

Dependent #7
TD

Dependent #1

Dependent #2

Dependent #3

Dependent #4

Dependent #5

Dependent #6

Dependent #7
RBC

Dependent #1

Dependent #2

Dependent #3

Dependent #4

Dependent #5

Dependent #6

Dependent #7
Scotia

Dependent #1

Dependent #2

Dependent #3

Dependent #4

Dependent #5

Dependent #6

Dependent #7


In [1]:
#Concatenate Dataframes

cibc_neural1_r2
cibc_neural1_r2
cibc_neural1_mse
bmo_neural1_r2
bmo_neural1_mse
rbc_neural1_r2
rbc_neural1_mse
td_neural1_r2
td_neural1_mse
scotia_neural1_r2
scotia_neural1_mse
Neural1_mse = pd.concat([cibc_neural1_mse,bmo_neural1_mse,rbc_neural1_mse,td_neural1_mse,scotia_neural1_mse])
Neural1_linear_r2 = pd.concat([cibc_neural1_r2,bmo_neural1_r2,rbc_neural1_r2,td_neural1_r2,scotia_neural1_r2])

NameError: name 'cibc_neural1_r2' is not defined